In [ ]:
# Databricks notebook: Download artifacts for latest run of a given task of THIS job
# ---------------------------------------------------------------------------------

# PARAMETERS
# ----------
dbutils.widgets.text("TASK_KEY", "")               # task_key (task name) within this job
dbutils.widgets.text("SOURCE_PATH", "")            # subfolder inside artifacts, empty for root
dbutils.widgets.text("DEST_PATH", "")   # local/DBFS dest on this cluster

In [ ]:
TASK_KEY = dbutils.widgets.get("TASK_KEY").strip()
SOURCE_PATH = dbutils.widgets.get("SOURCE_PATH").strip()
DEST_PATH = dbutils.widgets.get("DEST_PATH").strip()

In [ ]:
# Import Databricks SDK
from databricks.sdk import WorkspaceClient
import json

In [ ]:
# 1. Get JOB_ID of *this* notebook's job
# --------------------------------------
ctx_json_str = dbutils.notebook.entry_point.getDbutils().notebook().getContext().safeToJson()
print(type(ctx_json_str))
ctx_dict = json.loads(ctx_json_str)
print(ctx_dict)

# 2. Get job_id from the dict
JOB_ID = ctx_dict['attributes'].get("multitaskParentRunId")

print(f"Current job id (JOB_ID) = {JOB_ID}")

In [ ]:
all_args = dbutils.notebook.entry_point.getCurrentBindings()

print(all_args)

In [ ]:
# Create client using the job's identity (service principal / compute identity)
w = WorkspaceClient()

# Get run info (Jobs API 2.1)
run = w.jobs.get_run(run_id=int(JOB_ID))

# `run` is a JobsGetRunResponse; `run.tasks` is a list of RunTask instances
tasks = run.tasks or []

# Pretty-print equivalent to `jq .tasks`
print(json.dumps([t.as_dict() for t in tasks], indent=2))